In [1]:
from azureml.core import Workspace, Experiment, Environment

experiment_name = 'udacity-project'
environment = 'AzureML-sklearn-1.0-ubuntu20.04-py38-cpu'

#subscription = '5a4ab2ba-6c51-4805-8155-58759ad589d8'
#rg = 'aml-quickstarts-190075'
#workspace = 'quick-starts-ws-190075'

#ws = Workspace.get(name=workspace, subscription_id=subscription, resource_group=rg)
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name=experiment_name)

env = Environment.get(workspace=ws, name=environment)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (msrest 0.6.19 (c:\users\aldam\anaconda3\lib\site-packages), Requirement.parse('msrest>=0.6.21'), {'azure-mgmt-keyvault', 'azure-mgmt-resource', 'azure-mgmt-storage', 'azure-mgmt-containerregistry'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (msrest 0.6.19 (c:\users\aldam\anaconda3\lib\site-packages), Requirement.parse('msrest>=0.6.21'), {'azure-mgmt-keyvault', 'azure-mgmt-resource', 'azure-mgmt-storage', 'azure-mgmt-containerregistry'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (msrest 0.6.19 (c:\users\aldam\anaconda3\lib\site-packages), Requirement.parse('msrest>=0.6.21'), {'azure-mgmt-keyvault', 'azure-mg

Workspace name: quick-starts-ws-190075
Azure region: southcentralus
Subscription id: 5a4ab2ba-6c51-4805-8155-58759ad589d8
Resource group: aml-quickstarts-190075


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_name= 'cpu-compute-trn'
compute_size= 'Standard_D2_V2'

try:
    cpu_cluster_compute_target = ComputeTarget(workspace=ws, name=compute_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=compute_size, max_nodes=4)
    cpu_cluster_compute_target = ComputeTarget.create(ws, compute_name, compute_config)
cpu_cluster_compute_target.wait_for_completion(show_output=True)

print(cpu_cluster_compute_target.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2022-03-26T21:02:23.765000+00:00', 'errors': None, 'creationTime': '2022-03-26T21:02:19.791254+00:00', 'modifiedTime': '2022-03-26T21:02:23.354653+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import choice
from azureml.core import ScriptRunConfig, Environment
import os

# Specify parameter sampler
#ps  = ### YOUR CODE HERE ###
ps=RandomParameterSampling({
    "C": choice(100, 10, 1.0, 0.1, 0.01),
    "max_iter": choice(range(50,200))}
)

# Specify a Policy
#policy = ### YOUR CODE HERE ###
policy = BanditPolicy(slack_factor = 0.1, 
                      evaluation_interval=1, 
                      delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
#est = ### YOUR CODE HERE ###
est = SKLearn(source_directory='.',
            entry_script='train.py',
            compute_target=cpu_cluster_compute_target,
            environment_definition = env)

#est = ScriptRunConfig(source_directory=".",
#                     script="train.py",
#                     compute_target=cpu_cluster_compute_target,
#                     environment=env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
#hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps, 
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     max_total_runs=12,
                                     max_concurrent_runs=4)

In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [11]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
#parameter_values = best_run.get_details()['runDefinition']['Arguments']

#print('Best Run Id: ', best_run.id)
#print('\n Accuracy:', best_run_metrics['accuracy'])
#print('\n C:',parameter_values[0])
#print('\n max_iter:',parameter_values[1])

os.makedirs("./outputs", exist_ok=True)
joblib.dump( value=best_run.id, filename='outputs/bes_run_model.joblib')
print('Model has been sucessfully saved!')

Model has been sucessfully saved!


In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [13]:
from train import clean_data

# Use the clean_data function to clean your data.
ds.to_pandas_dataframe()
x, y = clean_data(ds)

In [16]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='regression',
    primary_metric='Accuracy',
    training_data=x,
    label_column_name='y',
    n_cross_validations=5)




In [17]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

exp = Experiment(ws, 'autoML')
run = exp.submit(config=automl_config, show_output=True)

The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Performing interactive authentication. Please follow the instructions on the terminal.


KeyboardInterrupt: 